In [2]:
%load_ext autoreload
%autoreload 2
import sys
import collections
import numpy as np
# sys.path.append("/private/home/alexandrerame/domainbedv2/")
# sys.path.append("/private/home/alexandrerame/slurmconfig/notebook/data")

sys.path.append("/home/rame/domainbedv2/")
sys.path.append("/home/rame/ExpansionNet_v2//")

from domainbed.codeplot import plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from eval.meteor.meteor import Meteor
from eval.meteor.reinforce_meteor import Meteor as ReinforceMeteor

In [4]:
from nltk.tokenize.treebank import TreebankWordTokenizer
from eval.tokenizer.ptbtokenizer import PTBTokenizer

from nltk.tokenize.stanford

In [15]:
s = '''Good muffins cost $3.88\nin New York,  Please buy me\ntwo of them.\nThanks.'''
gts = {1: [{"caption": s}]}

In [16]:
tokenizer = PTBTokenizer()
print(tokenizer.tokenize(gts))

{1: ['good muffins cost $ 3.88 in new york please buy me two of them thanks']}


In [20]:
PUNCTUATIONS = ["''", "'", "``", "`", "-LRB-", "-RRB-", "-LCB-", "-RCB-", \
        ".", "?", "!", ",", ":", "-", "--", "...", ";"]

In [26]:
from eval.tokenizer.pythontokenizer import PythonTokenizer

In [29]:
pt = PythonTokenizer()
print(pt.tokenize(gts))

Using NLTK tokenizer
{1: ['good muffins cost $ 3.88 in new york please buy me two of them thanks']}


In [22]:
gts_final_tokenized_captions_for_image = {}
for k, vs in gts.items():
    if not k in gts_final_tokenized_captions_for_image:
        gts_final_tokenized_captions_for_image[k] = []
    for v in vs:
        tokenized_caption = " ".join(t.tokenize(v['caption']))
        tokenized_caption = ' '.join([w for w in tokenized_caption if w not in PUNCTUATIONS]).lower()
        gts_final_tokenized_captions_for_image[k].append(tokenized_caption)

In [23]:
gts_final_tokenized_captions_for_image

{1: ['good muffins cost $ 3.88 in new york please buy me two of them. thanks']}

In [2]:
import nltk


In [1]:
from nltk.tokenize.treebank import TreebankWordTokenizer

In [5]:
t = TreebankWordTokenizer()
t.tokenize(s)

['Good',
 'muffins',
 'cost',
 '$',
 '3.88',
 'in',
 'New',
 'York.',
 'Please',
 'buy',
 'me',
 'two',
 'of',
 'them.',
 'Thanks',
 '.']

In [ ]:
gts = [""]

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/rame/nltk_data...


True

In [5]:
"swin_transf.norm.weight/bias".lstrip("swin_transf.")

'orm.weight/bias'

In [8]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/rame/nltk_data...


True

In [17]:
nltk.translate.meteor_score.meteor_score(
    ["this is an apple".split(" "), "that is an apple".split(" ")],
    "an apple on this tree".split(" ")
)

0.6233062330623306

In [10]:
def meteor(reference, hypothesis, alpha=0.5):
    reference_unigrams = Counter(reference.split())
    hypothesis_unigrams = Counter(hypothesis.split())
    reference_bigrams = Counter(zip(reference.split()[:-1], reference.split()[1:]))
    hypothesis_bigrams = Counter(zip(hypothesis.split()[:-1], hypothesis.split()[1:]))

    precision = (
        alpha * precision_n(reference_unigrams, hypothesis_unigrams, 1) +
        (1 - alpha) * precision_n(reference_bigrams, hypothesis_bigrams, 2)
    )
    recall = (
        alpha * recall_n(reference_unigrams, hypothesis_unigrams, 1) +
        (1 - alpha) * recall_n(reference_bigrams, hypothesis_bigrams, 2)
    )

    if precision == 0 or recall == 0:
        return 0

    harmonic_mean = (1 / precision + 1 / recall) / 2
    unigram_penalty = 0.5 * (1 - len(reference_unigrams) / len(hypothesis_unigrams))

    meteor_score = harmonic_mean * (1 - unigram_penalty)

    return meteor_score

In [11]:
from collections import Counter

In [12]:
def precision_n(reference, hypothesis, n):
    if not hypothesis:
        return 0

    intersection = 0
    for ngram in hypothesis:
        if ngram in reference:
            intersection += 1
    return intersection / len(hypothesis)


def recall_n(reference, hypothesis, n):
    if not reference:
        return 0

    intersection = 0
    for ngram in hypothesis:
        if ngram in reference:
            intersection += 1
    return intersection / len(reference)

In [13]:
class Meteor:

    def __init__(self):
        # vrama91: updated the value below based on discussion with Hovey
        self.alpha = 0.5

    def calc_score(self, hypo, refs):
        score = 0
        for ref in refs:
            score += meteor(reference=ref, hypothesis=hypo, alpha=self.alpha)
        return score / len(refs)

In [14]:
m = Meteor()


In [16]:
m.calc_score(
    refs=["this is an apple", "that is an apple"],
    hypo="an apple on this tree"
)

2.17710407239819